# Scott Breitbach
## 20-March-2021
## DSC550, Week 2

In [26]:
import pandas as pd
import unicodedata
import sys
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk import pos_tag
from nltk import word_tokenize
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

## 1) Read the *controversial-comments.jsonl* file and pre-process the text. 

In [2]:
allCommentsDF = pd.read_json("controversial-comments\controversial-comments.jsonl", lines=True)
allCommentsDF

,con,txt
0,0,Well it's great that he did something about th...
1,0,You are right Mr. President.
2,0,You have given no input apart from saying I am...
3,0,I get the frustration but the reason they want...
4,0,I am far from an expert on TPP and I would ten...
...,...,...
949995,0,I genuinely can't understand how anyone can su...
949996,0,"As a reminder, this subreddit [is for civil di..."
949997,0,K. Don't explain why or anything.
949998,0,[deleted]


Grab a sample of the set to work with:

In [3]:
commentsDF = allCommentsDF.sample(100)

#### A) Convert all text to lowercase letters.

In [4]:
commentsDF.txt = commentsDF['txt'].str.lower()

In [5]:
commentsDF.head()

,con,txt
436177,0,http://www.nber.org/papers/w9873
743268,0,i don't know where to start. i will lay out so...
139742,0,"yes very slim, there is not money in it like t..."
564772,1,[deleted]
744226,0,did republicans esentenially get rid of the co...


#### B) Remove all punctuation from the text.

In [6]:
punctuation = dict.fromkeys(i for i in range(sys.maxunicode) 
                            if unicodedata.category(chr(i)).startswith('P'))

In [7]:
commentsDF.txt = [string.translate(punctuation) for string in commentsDF.txt]

In [8]:
commentsDF.head()

,con,txt
436177,0,httpwwwnberorgpapersw9873
743268,0,i dont know where to start i will lay out some...
139742,0,yes very slim there is not money in it like th...
564772,1,deleted
744226,0,did republicans esentenially get rid of the co...


#### C) Remove stop words.

In [9]:
# import nltk
# nltk.download('stopwords')

In [10]:
stop_words = stopwords.words('english')

def removeStopWords(string):
    tokenized_words = word_tokenize(string)
    return [word for word in tokenized_words if word not in stop_words]

In [11]:
commentsDF.txt = commentsDF.txt.apply(lambda x: removeStopWords(x))

In [12]:
commentsDF.head()

,con,txt
436177,0,[httpwwwnberorgpapersw9873]
743268,0,"[dont, know, start, lay, facts, probably, know..."
139742,0,"[yes, slim, money, like, used, cole, declining..."
564772,1,[deleted]
744226,0,"[republicans, esentenially, get, rid, comittee..."


#### D) Apply NLTK's PorterStemmer.

In [13]:
porter = PorterStemmer()

In [14]:
commentsDF.txt = commentsDF.txt.apply(lambda x: [porter.stem(word) for word in x])

In [15]:
commentsDF.head()

,con,txt
436177,0,[httpwwwnberorgpapersw9873]
743268,0,"[dont, know, start, lay, fact, probabl, know, ..."
139742,0,"[ye, slim, money, like, use, cole, declin, sin..."
564772,1,[delet]
744226,0,"[republican, esenteni, get, rid, comitte, fili..."


## 2) Get text into a usable form for model-building.

#### A) Convert each text entry into a word-count vector.
See sections 5.3 & 6.8 in the *Machine Learning with Python Cookbook*

In [16]:
count = CountVectorizer()

In [17]:
# commentsDF['WCV'] = commentsDF.txt.apply(lambda x: count.fit_transform(x).toarray())

Converting to word-count vector threw an error when the 'txt' field contained an empty list. This function should return an empty list when coming across this error.

In [18]:
# def wordCountVector(wordList):
#     try:
#         array = count.fit_transform(wordList).toarray()
#         return array
#     except:
#         return []

In [19]:
# commentsDF['WCV'] = commentsDF.txt.apply(lambda x: wordCountVector(x))
# commentsDF.head()

Apparently this needs to be in one large matrix so let's try it again:

In [20]:
# Convert all the text data into a list of strings, 
# with each tweet as one string in the list

text_data, string = [], " "

for text in commentsDF.txt:
    text_data.append(string.join(text))

In [21]:
# Word-count vector as a DataFrame
wordCountVector = pd.DataFrame(count.fit_transform(text_data).toarray(), columns=count.get_feature_names())
wordCountVector

,10,100,10000,1020,1128,12,12012016,15,15yearold,2016,...,wouldnt,write,wrong,wrote,ye,yea,yeah,year,your,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Word-count vector as a sparse matrix
sparseWCV = count.fit_transform(text_data)
sparseWCV

<100x914 sparse matrix of type '<class 'numpy.int64'>'
	with 1502 stored elements in Compressed Sparse Row format>

#### B) Convert each text entry into a part-of-speech tag vector.
See section 6.7 in the *Machine Learning with Python Cookbook*

In [23]:
nltk.pos_tag(commentsDF.txt.iloc[0])[:5]

[('httpwwwnberorgpapersw9873', 'NN')]

In [24]:
commentsDF['PoS'] = commentsDF.txt.apply(lambda x: [tag for word, tag in nltk.pos_tag(x)])

In [25]:
commentsDF.head()

,con,txt,PoS
436177,0,[httpwwwnberorgpapersw9873],[NN]
743268,0,"[dont, know, start, lay, fact, probabl, know, ...","[NN, VBP, VBP, VB, NN, NN, VBP, MD, VB, VB, NN..."
139742,0,"[ye, slim, money, like, use, cole, declin, sin...","[NN, JJ, NN, IN, NN, JJ, NN, VBD, CD, NN, JJ, ..."
564772,1,[delet],[NN]
744226,0,"[republican, esenteni, get, rid, comitte, fili...","[JJ, NN, VB, JJ, NN, NN, NN]"


#### C) Convert each entry into a term frequency-inverse document frequency (**tfidf**) vector.
See section 6.9 in the *Machine Learning with Python Cookbook*

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
# tfidf vector as a Dataframe:
tfidfVector = pd.DataFrame(tfidf.fit_transform(text_data).toarray(), columns=tfidf.get_feature_names())
tfidfVector

In [ ]:
# tfidf vector as a sparse matrix:
sparseTfidf = tfidf.fit_transform(text_data)
sparseTfidf

## **Follow-Up Question**

### For the three techniques in problem 2 above, give an example where each would be useful.

#### A) Word-count vector

#### B) Part-of-speech vector

#### C) tfdif vector